In [ ]:
import numpy as np
import pandas as pd
import math
import scipy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import os

%matplotlib inline  
pd.options.display.mpl_style = 'default'
pylab.rcParams['figure.figsize'] = 12, 6 

In [ ]:
ge = get("GE")
ibm = get('IBM')
msft = get('MSFT')
ebay = get("EBAY")
swks = get("SWKS")
cvx = get("CVX")
mu = get('MU')
gild = get("GILD")
aapl = get('AAPL')

In [1]:
def get(ticker):
    temp_df = pd.read_csv("https://www.quandl.com/api/v3/datasets/YAHOO/"+ticker+".csv")
    temp_df.index = pd.to_datetime(temp_df.Date)
    temp_df = temp_df.sort()
    return temp_df

In [8]:
portfolio = pd.DataFrame()
portfolio=portfolio.append(aapl.Close)
portfolio=portfolio.append(ge.Close)
portfolio=portfolio.append(mu.Close)
portfolio=portfolio.append(aapl.Close)


portfolio = portfolio.transpose()
portfolio.columns = ['aapl', 'ge', 'none', 'mu']
portfolio = portfolio[['aapl', 'ge', 'mu']]
portfolio = portfolio.diff()
#portfolio

In [12]:
risk = portfolio_risk(portfolio, [.3, .3, .4])
return1 = np.matrix(portfolio.mean())
return1 * np.matrix([.3, .3, .4]).transpose()
sharpe = return1 * np.matrix([.3, .3, .4]).transpose() / risk
sharpe * 240


matrix([[ 0.06976648]])

In [9]:
def portfolio_risk(price_df, weight_list):
    #Take in a PRICE_DF with columns being each stock's daily log_returns
    #Take in WEIGHT_LIST with each list entry corresponding to the respective price_df
    #column's weight in the portfolio
    #PRICE_DF should be a concatenation of return series from return_series()
    if (sum(weight_list) != 1):
        print("EROR: Weights do not sum to 1");
        return None
    cov_matrix = np.matrix(price_df.cov())
    weight_t = np.matrix(weight_list)
    weight = weight_t.transpose()
    variance = weight_t * cov_matrix * weight
    return variance
    

In [28]:
portfolio_risk(returns[['ge', 'msft', 'aapl', 'mu']], [.2, .2, .4, .2])



matrix([[ 95.25385787]])

In [43]:
def return_series(price_df, col):
    #Take in a price_df for one security
    #Return a series of the daily log returns of the price_df[col] column
    price_df.index = pd.to_datetime(price_df.Date)
    price_df = price_df.sort()
    rets = np.log(price_df[col]).diff()
    return (rets)


In [45]:
return_series(aapl, "mu")

Date
2000-01-03         NaN
2000-01-04   -0.088080
2000-01-05    0.014530
2000-01-06   -0.090514
2000-01-07    0.046280
2000-01-10   -0.017744
2000-01-11   -0.052505
2000-01-12   -0.061846
2000-01-13    0.104070
2000-01-14    0.037403
2000-01-18    0.034254
2000-01-19    0.024942
2000-01-20    0.063072
2000-01-21   -0.019460
2000-01-24   -0.046548
...
2015-08-10    0.035712
2015-08-11   -0.053441
2015-08-12    0.015302
2015-08-13   -0.000781
2015-08-14    0.007010
2015-08-17    0.010295
2015-08-18   -0.005649
2015-08-19   -0.012872
2015-08-20   -0.020733
2015-08-21   -0.063113
2015-08-24   -0.025279
2015-08-25    0.005994
2015-08-26    0.055770
2015-08-27    0.029021
2015-08-28    0.003271
Name: mu, Length: 3939

In [210]:
def strat1(ticker):
    def get(ticker):
        temp_df = pd.read_csv("https://www.quandl.com/api/v3/datasets/YAHOO/"+ticker+".csv")
        temp_df.index = pd.to_datetime(temp_df.Date)
        temp_df = temp_df.sort()
        return temp_df
    ret_df = get(ticker)
    ret_df['ret'] = ret_df.Close.shift(-1) - ret_df.Close
    return_df = ret_df
    return_df['day1'] = np.where(return_df.ret > return_df.ret.shift(1), 1, 0)
    return_df['five_days'] = np.where(return_df.day1 + return_df.day1.shift(1) +
                                 return_df.day1.shift(2) + return_df.day1.shift(3) +
                                 return_df.day1.shift(4) + return_df.day1.shift(5) == 5, 1, 0)
    return_df['trade_yes'] = np.where(return_df.five_days == 1, 1, 0)
    return_df['ret_long'] = np.where(return_df.trade_yes == 1, return_df.ret + return_df.ret.shift(-1) +
                                 return_df.ret.shift(-2) + return_df.ret.shift(-3) +
                                 return_df.ret.shift(-4) + return_df.ret.shift(-5), 0)
    return_df['ret_short'] = -1 * return_df.ret_long
    return_df = return_df[return_df.index.year > 2000]
    length = return_df.ret_long.size / 15
    sharpe_df = return_df.ret_long.cumsum()[::length].shift(-1) - return_df.ret_long.cumsum()[::length]
    yearly_sharpe = sharpe_df.mean() / sharpe_df.std()
    return_df.ret_long.cumsum().plot(title = ticker+ "\n--Five Day Trade--\nSharpe = " + str(yearly_sharpe));
    return return_df